# 숙명여대 학과정보수집

###  #01.패키지 참조

In [1]:
# 필요한 모듈 참조
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

#### #02.웹 페이지 코드 수집


In [4]:
## 수집할 컨텐츠가 있는 웹페이지 주소
## 물음표 뒤에오는 걸 쿼리 스트링 : 변수이름 = 값 & 변수이름 = 값
## https : 통신방식에 대한 약속, 

#      프로토콜  / 도메인      /파일 경로/ (? 뒤에 변수이름 =값 & 변수이름 = 값 ) ->쿼리스트링
#브라우저 버전정보
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

## 접속객체 생성
session=requests.Session()

# 접속객체에 header정보 삽입
session.headers.update({
    "Referer": "",
    "User-Agent": userAgent,

})



# while 문 써서 count함수 구현하기 공부 ㄱㄱ


#### #03. 응답결과로부터 내용 추출하기

In [5]:
## 대학번호 리스트 정의 --> url을 분석해서 얻은값 
## 전체 학부 + 전체 전공까지 for문으로 돌려서 크롤링

clist = [1011,1023,1030,1041,1046,1052,1054,1057,1062,1064,1070,1073,1076]

In [9]:
# 수집할 정보를 저장할 빈 리스트 

학과목록_리스트 =[]
for c in clist :
    ## 접속객체를 통해 api접속
    r= session.get(url)
    url = "https://www.sookmyung.ac.kr/sookmyungkr/%d/subview.do" % c
    # 접속에 실패했을 때 

    if r.status_code!=200 :
        # 200이 아니면 에러코드와 에러메세지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code,r.reason)
        # 에러를 강제로 생성시킨다.
        raise Exception(msg)
        
    # 인코딩 형식 지정
    r.encoding = 'utf-8'
    ## 전체 html파일
    soup = BeautifulSoup(r.text)


    college_list = soup.select(".college_list")
    college_list
    for item in college_list :
        # print(item)
        # print("-"*90)
        ## 학과이름 
        nameEl = item.select("h5")
        name = nameEl[0].text.strip()
        # print(name)
        # print("-"*90)
        ## 상세보기 url
        viewEl = item.select(".view")
        view = viewEl[0].attrs['href']

        
        # print(view)
        ## 가져온 주소는 /sookmyungkr/1012/subview.do 인데 외부에서 직접 접근할 때는 풀 주소가 필요함
        ## https://www.sookmyung.ac.kr/sookmyungkr/1011/subview.do 필요함
        if view.find("https://www.sookmyung.ac.kr") == -1 :
            view = "https://www.sookmyung.ac.kr" + view
        # print(view)
        # print("-"*90)
        ## 학과소개 pdf링크
        pdfEl = item.select('.info')
        # print(infoEl)
        # print("-"*90)
        pdf = pdfEl[0].attrs['href']

        ## 학과 홈페이지 
        homepageEl = item.select('.homepage')
        # print(homepageEl)
        # print("-"*90)
        homepage = homepageEl[0].attrs['href']
        # print(homepage)
        ## 수집한 값들을 딕셔너리에 묶기 
        college_dict = {
            '학과이름' : name,
            '상세페이지' : view,
            '홈페이지' : homepage
        }
        # -------------------------------------------------

        # 상세보기 url을 새롭게 수집(시작)

        # -------------------------------------------------
        
        r= session.get(view)
        if r.status_code!=200 :
            # 200이 아니면 에러코드와 에러메세지 출력
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code,r.reason)
            # 에러를 강제로 생성시킨다.
            print(msg)
            continue ## 학과홈페이지 로딩하다가 에러나면 그냥 넘기기 위해서 continue
        r.encoding = 'utf-8'

        detailSoup = BeautifulSoup(r.text)
        # print(detailSoup)
        ## 같은 구조를 갖느 전화번호, 팩스번호, 위치, 이메일주소를 가져온다. 
        infoEl = detailSoup.select(".college_info_data dl")
        # print(infoEl)
        # print("-"*90)
        for info in infoEl :
            dt = info.select("dt")[0].text.replace(":",'').strip()
            # print(dt)
            dd =info.select("dd")[0].text.replace(":","").strip()
            # print(dd)
            # print("-"*90)
            college_dict[dt] = dd
        # -------------------------------------------------

        # 상세보기 url을 새롭게 수집 (끝)

        # -------------------------------------------------
        ## 수집한 값들을 리스트에 append 
        
        ## pdf 파일 다운로드 + 저장 -> url 저장하고있는 변수는 pdf
        print(pdf)
        r=session.get(pdf, stream = True)
        if pdf.find(".pdf") > -1 : ## 없으면pdf가 없으면 -1 리턴하는데 .pdf 있어야 다운 받으셈 
            #https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_01_2022.pdf 다운받는 링크임
            r=session.get(pdf, stream = True)
            if r.status_code == 200 :
                r.encoding = "utf-8"
                with open("%s.pdf" % name,'wb') as f :
                    f.write(r.raw.read())
            
        학과목록_리스트.append(college_dict)
# print(학과목록_리스트)
df = DataFrame(학과목록_리스트)
df.to_excel("result.xlsx")
df

/sookmyungkr/1071/subview.do
/sookmyungkr/1072/subview.do
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_01_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_02_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_03_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_04_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_05_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_06_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_07_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_08_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_09_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_10_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_11_2022.pdf
https://admission.sookmyung.ac.kr/enter/pdf/major/Sookmyung_Major_12_2022.pdf
https:

KeyboardInterrupt: 